In [1]:
using Interact
using LCMGL
using ForwardDiff
using RigidBodyDynamics
using DataFrames
using SimpleGradientDescent
using Ipopt

In [2]:
import Flash
reload("Flash")

adding: /home/rdeits/locomotion/explorations/point-cloud-signed-distance/packages/v0.5/DrakeVisualizer/src/lcmtypes to the python path


In [3]:
squished = Flash.DepthData.read_point_cloud(open("data/squishable_squished_xyzrgb.txt"))

PointCloud with origin: [1.37689,0.775602,1.4848] containing 25164 points

In [4]:
Flash.DepthData.render_lcmgl(squished)

In [5]:
unsquished = Flash.DepthData.read_point_cloud(open("data/squishable_unsquished_xyzrgb.txt"))

PointCloud with origin: [1.38246,0.768824,1.48581] containing 25571 points

In [6]:
Flash.DepthData.render_lcmgl(unsquished)

In [7]:
model = Flash.Models.squishable()
state = Flash.ManipulatorState(model)
Flash.draw(state)

DrakeVisualizer.Visualizer(DrakeVisualizer.Robot(DrakeVisualizer.Link[DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 302xFixedSizeArrays.Point{3,Float64},     faces: 600xGeometryTypes.Face{3,Int64,0}, )
,CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))],"skin_1")]),1,PyLCM.LCM(PyObject <LCM object at 0x7f016ff7b510>))

In [8]:
point_cloud = unsquished
Flash.DepthData.render_lcmgl(point_cloud)
sensed_points = [p.position for p in point_cloud.points]

cost = Flash.CostFunctor(model, sensed_points)

wrapped_cost = x -> begin
    x[1:4] = x[1:4] ./ norm(x[1:4])
    c = cost(x)
    Flash.draw(get(cost.state), false)
    c
end

(::#16) (generic function with 1 method)

In [9]:
cost_and_gradient! = Flash.CostAndGradientFunctor(wrapped_cost)

(::#29) (generic function with 1 method)

In [10]:
trials = []


function test_gradient_descent()
    for i = 1:1
        xs = []

        # We'll initialize our estimate with a uniformly random pair
        # of joint angles
        x_estimated = Flash.flatten(Flash.ManipulatorState(model))
        x_estimated[5:7] = [.5, 0, .5]
        g = zeros(x_estimated)

        # Now we use gradient descent to recover the joint angles
        for i = 1:50
            push!(xs, x_estimated)

            # Evaluate the gradient of the point values w.r.t. the
            # joint angles:
            ci = cost_and_gradient!(g, x_estimated)

            if any(isnan, g)
                @show x_estimated
                error("NaN in gradient")
            end
            
            # Descend the gradient
            step = -ci ./ g
            step[1:7] = sign(step[1:7]) .* min.(0.01, abs.(g[1:7]))
            step[8:end] = sign(step[8:end]) .* min.(1e-5, abs.(g[8:end]))
            x_estimated += step

            # Descend the gradient
#             x_estimated -= 0.0000002 * gi
            
            # TODO: generalize this
            x_estimated[1:4] = x_estimated[1:4] / norm(x_estimated[1:4])
        end
        push!(trials, DataFrame(x=xs))
    end
end

test_gradient_descent (generic function with 1 method)

In [11]:
test_gradient_descent()
@time test_gradient_descent()
state = Flash.ManipulatorState(model)
Flash.unflatten!(state, trials[1][:x][end])
Flash.draw(state)

  7.016855 seconds (44.94 M allocations: 3.768 GB, 6.62% gc time)


DrakeVisualizer.Visualizer(DrakeVisualizer.Robot(DrakeVisualizer.Link[DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 298xFixedSizeArrays.Point{3,Float64},     faces: 592xGeometryTypes.Face{3,Int64,0}, )
,CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))],"skin_1")]),1,PyLCM.LCM(PyObject <LCM object at 0x7f016ff7b570>))

In [12]:
x0 = Flash.flatten(Flash.ManipulatorState(model))
num_vars = length(x0)
solver = IpoptSolver(print_level=1, max_iter=20)
cost_functor = wrapped_cost
gradient_functor = Flash.CostAndGradientFunctor(Flash.CostFunctor(model, sensed_points))
m = getmodel(GradientDescent(cost_functor, gradient_functor), num_vars, solver)
setwarmstart!(m, x0);

In [13]:
optimize!(m)
x = getsolution(m)
state = Flash.ManipulatorState(model)
Flash.unflatten!(state, x)
Flash.draw(state)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



DrakeVisualizer.Visualizer(DrakeVisualizer.Robot(DrakeVisualizer.Link[DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 167015xFixedSizeArrays.Point{3,Float64},     faces: 332384xGeometryTypes.Face{3,Int64,0}, )
,CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))],"skin_1")]),1,PyLCM.LCM(PyObject <LCM object at 0x7f016ff7b6c0>))